In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


Bad key text.latex.unicode in file /home/gmartini2019/miniconda3/envs/bruce_environment/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/gmartini2019/miniconda3/envs/bruce_environment/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/gmartini2019/miniconda3/envs/bruce_environment/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplsty

In [2]:
# Load image
try:
    image = cv2.imread('/mnt/c/Users/marti/OneDrive/Desktop/chart_detection/782234.png')
    if image is None:
        raise ValueError('Failed to load image')
except Exception as e:
    print('Error:', e)
    exit()

# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 100, 200)

# Find contours in the image
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through each contour
for i, contour in enumerate(contours):
    # Calculate the contour area
    area = cv2.contourArea(contour)
    # If the area is small, skip the contour
    if area < 10000:
        continue
    # Create a mask from the contour and apply it to the original image
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, contours, i, (255, 255, 255), cv2.FILLED)
    # Create a 4-channel image with an alpha channel
    chart = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
    chart[:, :, :3] = image
    chart[:, :, 3] = mask
    # Save the chart as a new image with a transparent background
    cv2.imwrite(f'chart{i}_reports_demo.png', chart)

In [5]:
from wand.image import Image

def convert_pdf_to_images(pdf_path):
    with Image(filename=pdf_path, resolution=300) as source:
        images = []
        for i, page in enumerate(source.sequence):
            with Image(page) as img:
                img.format = 'png'
                img.alpha_channel = 'remove'
                img_name = f"image_from_pdf_number_{i}.png"
                img.save(filename=img_name)
                images.append(img.make_blob())
    return images

pages = convert_pdf_to_images('/mnt/c/Users/marti/OneDrive/Desktop/chart_detection/pdf_test1.pdf')